### Importing the necessary libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import pydicom as dcm
import time
import tensorflow as tf
from matplotlib import pyplot as plt

## normalising the CT scans
I wanted to normalise the CT scans to speed up training. At first, I wanted to compute the mean and std of the first image and then normalize the whole dataset using it; after all, a CT scan of a lung should be similar to another lung, right? But decided against it and decided that maybe computing the global mean and std might be worth the effort. So I wrote the following code.

When I suspect the code is gonna take a long time I insert a print statement somewhere so that I know the code works. I coded the print statement so as to print the mean of images up to the patients that I opened their CT scans. I noticed that the the means starts out high and then goes down up to a negative value then goes up again, and the patern continues as the for loop goes on. I thought there was something wrong with my code.

In [ ]:
data_path = "../input/osic-pulmonary-fibrosis-progression"
train_path = data_path+"/train/"
patients = os.listdir(train_path)
def load_dcm(path):
    dcm_data = dcm.dcmread(path)
    pixels = dcm_data.pixel_array
    mean, std = np.mean(pixels), np.std(pixels)
    return mean, std
    # return (pixels-mean)/std, dcm_data.get("ImagePositionPatient")
mean = []
std = []
for directory, _, files in os.walk(train_path):
    for file in files:
        try:
            m, s = load_dcm(os.path.join(directory, file))
            mean.append(m)
            std.append(s)
        except Exception:
            print(os.path.join(directory, file))
    print(np.mean(mean))

I tried to plot the distribution of the mean and turned out there are about two major clusters of means.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
ax.hist(mean, bins=100)

ploting the std distribution yielded 3 clusters.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
ax.hist(std, bins=100)

So how do we go about this? do we normalize the the CT scans per cluster they belong to, or should we normalize each image with its own mean and std or do we just ignore this and just notmalize using the std and mean of the whole dataset?